In [ ]:
df_embedding = sqlContext.read.csv('gs://pvtrough_asia_south1/clustering/df_embedding_LM_large.csv', sep=',',
                         inferSchema=True, header=True)
df_embedding.take(1)

In [ ]:
from pyspark.sql.functions import array, col
feat_cols = [str(x) for x in range(0,768)]
df_embedding = df_embedding.select('hid', array([col(x) for x in feat_cols ]).alias('embedding'))
df_embedding.show()

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.functions import col, expr
k = 0
while k<=6:
    print(sampled_dates[k])
    n_days = 40
    st_date = sampled_dates[k] - datetime.timedelta(days=n_days+1)

    date_fmt = "%Y/%m/%d"

    dates = [(st_date + datetime.timedelta(days=i)) for i in range(n_days+1)]
    dates.sort()

    dates_str = [date.strftime(date_fmt) for date in dates]
    print(dates_str)
    # millisMin = dates[0].timestamp() * 1000
    # millisMax = (dates[-1] + datetime.timedelta(days=1)).timestamp() * 1000

    hashIdsWithFilter, newsMap = getNewsData(dates[0], sampled_dates[k])

    NIS_RAW_DATA_BASE_PATH = "gs://inshorts-segment-raw/data/segment-raw-v5/"

    path = get_raw_path(sampled_dates[k].strftime("%Y/%m/%d"))
    print(path)
    today_data = process_raw_data(path)
    today_data = today_data.filter(today_data.hashId.isin(hashIdsWithFilter))

    datestr = [d.strftime(date_fmt) for d in dates]
    paths = get_path(datestr, 'timeSpentFrontEvents')

    p_17 = 'gs://nis-segment-datasource-v3/processed/2023/02/17/timeSpentFrontEvents/*.parquet'
    if p_17 in paths: paths.remove(p_17)

    data = sqlContext.read.parquet(*paths)

    data = filter_app(data, app_name=None)
    data = filter_tenant(data, tenant='en')

    # data = data.filter((data.eventTimestamp > millisMin) & (data.eventTimestamp < millisMax))
    data = data.select(data.deviceId, data.overallTimeSpent, (F.split(data.hashId, '-')[0]).alias('hashId'))\
            .groupby('deviceId', 'hashId') \
            .agg(F.max('overallTimeSpent').alias('overallTimeSpent'))

    data = data.filter(data.hashId.isin(hashIdsWithFilter))

    data = data.union(today_data)
    data.cache()
    
#     newsMeanMap = defaultdict(lambda: (7, 1))
#     popularity(data, newsMeanMap)
    newsMeanMap = newsMeanMap_dict[sampled_dates[k].strftime("%Y_%m_%d")]

    data_filtered = data.filter(F.udf(lambda hashId, overallTimeSpent: overallTimeSpent > (2.5 * newsMeanMap[hashId][0]/newsMeanMap[hashId][1]), BooleanType())('hashId', 'overallTimeSpent'))
    
    # calculate idf_weight
    N = data_filtered.select('hashId').distinct().count()
    users = data_filtered.groupBy('hashId').agg(F.countDistinct('deviceId').alias('users'))
    idf_weights = users.withColumn('idf_weight', F.log(N / F.col('users')))
    
    # Multiply embeddings with IDF weights
    df_result = df_embedding.join(idf_weights, df_embedding.hid == idf_weights.hashId)
    df_result = df_result.withColumn('weighted_embedding', F.expr('transform(embedding, (x, i) -> x * idf_weight)'))
    
    
    df_result = df_result.select('hid','weighted_embedding')
    df_joined = data_filtered.join(df_result, data_filtered.hashId == df_result.hid , 'left')
    

    # assuming your dataframe is called df
    grouped_df = df_joined.groupBy("deviceId").agg(*[
        expr(f"avg(weighted_embedding[{i}]) as avg_{i}") for i in range(768)
    ])

    
    
    # assuming your dataframe is called grouped_df
    avg_cols = [col(f"avg_{i}") for i in range(768)]
    grouped_df = grouped_df.withColumn("topic_avg_embedding", array(*avg_cols))
    
    path_vec = "gs://pvtrough_asia_south1/clustering/device_vectors_roberta_idf1_AVG_large_"+sampled_dates[k].strftime("%Y_%m_%d")+"/"
    print(path_vec)
    
    grouped_df.select('deviceId', 'topic_avg_embedding').write.parquet(path=path_vec, mode='overwrite')
    
    k = k+1
#     def popularity(data, newsMeanMap):
#         Log("Computing news mean")
#         newsMeandf = data.groupby(data.hashId).agg({'overallTimeSpent' : 'sum', 'hashId' : 'count'}).toPandas()

#         for v in newsMeandf.values:
#             newsMeanMap[v[0]] = newsMeanMap[v[0]][0] + v[1],  newsMeanMap[v[0]][1] + v[2]

#     newsMeanMap = defaultdict(lambda: (7, 1))
#     popularity(data, newsMeanMap)

#     #run Storage
#     newsMeanMap_dict[sampled_dates[k].strftime("%Y_%m_%d")] = newsMeanMap.copy()
    
#     k=k+1

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import  pyspark.sql.functions as F
import  pyspark.sql.types as T
import pyspark.sql.window as W
import pandas as pd
import datetime

print(0)
import sys
from google.cloud import storage
import os
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import sqrt
import random
import numpy as np

from pyspark.sql import functions as F

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = SparkConf().setAll([('spark.jars.packages','com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.27.0')
                          ,('spark.executor.instances',"2")
                          ,('spark.dynamicAllocation.enabled',"false")
                          ])
sc = SparkContext.getOrCreate(conf=conf)
#Step 1: one time context generation
spark=SparkSession(sc)

bucket='pvtrough_asia_south1/tmp/tmpgcs_for_bq'
spark.conf.set('temporaryGcsBucket', bucket)
spark.conf.set("viewsEnabled","true")
spark.conf.set("materializationDataset",'tmp')

print (sc.applicationId)

prop_key=['power_user_flg','created_dt','days_active']

# spark.read.parquet()
stnd_jql_fmt = "%Y-%m-%d"
gcs_inp_fmt = "%Y/%m/%d"

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import  pyspark.sql.functions as F
import  pyspark.sql.types as T
import pyspark.sql.window as W
import pandas as pd
import datetime

print(0)
import sys
from google.cloud import storage
import os
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import sqrt
import random
import numpy as np

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
print(0)

0
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e115a80-22d6-4b09-a9b0-098bb82c2082;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.27.0 in central
:: resolution report :: resolve 166ms :: artifacts dl 2ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.27.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrie

application_1680918691099_1680
0
0
time: 637 µs (started: 2023-06-03 17:15:13 +00:00)


In [5]:
#Step 2 -repetitve for different dates
master_dt="2023-02-03"
#category metrics over past 7 days from above dt  are stored in below bucket
storage_buck="gs://pvtrough_asia_south1/tmp/deviceid_category_trend_notif"+master_dt.replace('-','_')
agg_query='''SELECT * FROM inshorts-1374.inshorts_scheduled.deviceid_category_trend 
WHERE event_date <= date_add(date('{0}'),INTERVAL 10 DAY) and event_date >= date('{0}') and upper(coalesce(appname,'UNKNOWN')) in ('UNKNOWN','INSHORTS')
and short_Views>=1
'''.format(master_dt)
#Note: there are cases where notification was sent(notif_shown) for a category but there was no short_views on it, as above table will contain all such records 
# we can choose categories with atleast one short view by  applying short_views>=1 filter

spark.read.format('bigquery').option('query',agg_query).load().coalesce(6).write.parquet(storage_buck,mode='overwrite')

print (storage_buck)



gs://pvtrough_asia_south1/tmp/deviceid_category_trend_notif2023_02_03
time: 2min 25s (started: 2023-06-03 17:22:38 +00:00)
